In [1]:
## Boiler plate code common to many notebooks.  See the TestFilesCommonCode.ipynb for details
from __future__ import print_function
%run TestFilesCommonCode.ipynb

SimpleITK Version: 0.9.1
Compiled: Sep 28 2015 10:07:41



In [2]:
import os
import glob
import sys

#\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/\/
#####################################################################################
#     Prepend the shell environment search paths
PROGRAM_PATHS = '/scratch/NAMICExternalProjects/release-20160523/bin'
#PROGRAM_PATHS = '/scratch/BS/release-BSR/bin'
PROGRAM_PATHS = PROGRAM_PATHS.split(':')
PROGRAM_PATHS.extend(os.environ['PATH'].split(':'))
os.environ['PATH'] = ':'.join(PROGRAM_PATHS)

CUSTOM_ENVIRONMENT=dict()

# Platform specific information
#     Prepend the python search paths
#PYTHON_AUX_PATHS = '/scratch/BS/BRAINSTools/AutoWorkup'
PYTHON_AUX_PATHS = '/scratch/SuperResolution/BRAINSSuperResolution/HCPWorkflows/'
PYTHON_AUX_PATHS = PYTHON_AUX_PATHS.split(':')
PYTHON_AUX_PATHS.extend(sys.path)
sys.path = PYTHON_AUX_PATHS

import SimpleITK as sitk
import nipype
from nipype.interfaces.base import CommandLine, CommandLineInputSpec, TraitedSpec, File, Directory
from nipype.interfaces.base import traits, isdefined, BaseInterface
from nipype.interfaces.utility import Merge, Split, Function, Rename, IdentityInterface
import nipype.interfaces.io as nio   # Data i/oS
import nipype.pipeline.engine as pe  # pypeline engine
from nipype.interfaces.freesurfer import ReconAll
from nipype.interfaces.ants import DenoiseImage
from nipype.interfaces.semtools import *

PYTHON_AUX_PATHS = '/scratch/wmql/tract_querier/tract_querier/'
PYTHON_AUX_PATHS = PYTHON_AUX_PATHS.split(':')
PYTHON_AUX_PATHS.extend(sys.path)
sys.path = PYTHON_AUX_PATHS

In [39]:
MatlabFilesDir = '/scratch/TESTS/IpythonNotebook/20160615_HCPWF/mainWF/test_wmql/VTPToMatlab'
# gs: gold standard
# in: input
#input_vtp_fn = '/scratch/TESTS/IpythonNotebook/20160615_HCPWF/mainWF/test_wmql/July31/WTV_query_cst.left.vtp'
input_vtp_fn = '/scratch/TESTS/IpythonNotebook/20160615_HCPWF/mainWF/Outputs/WMQL/NN_query_cst.right.vtp'
output_fn = os.path.join(MatlabFilesDir,'NN_cst.mat')

In [40]:
from tract_querier import tract_math, tractography
import numpy as np
import scipy.io

In [41]:
in_tractography = tractography.tractography_from_files(input_vtp_fn)
tracts = in_tractography.tracts()
pts = np.vstack(tracts)

In [42]:
scipy.io.savemat(output_fn, mdict={'NN': pts})

In [ ]:
def ComputeBhattacharyyaCoeficient(gs_bundle,sr_bundle):
    from tract_querier import tract_math, tractography
    import numpy as np
    #####
    def AlignFiber(tract):
        out_tract = tract
        vecs = np.diff(tract, axis=0)
        avg_vec = np.mean(vecs, axis=0)
        dots = np.array([ np.dot(avg_vec,[1,0,0]), np.dot(avg_vec,[0,1,0]), np.dot(avg_vec,[0,0,1]) ])
        idx = np.argmax(np.abs(dots))
        if (dots[idx] < 0):
            out_tract = tract[::-1]
        return out_tract
    #
    def InterpolateFiber(tract):
        new_tract = tract
        # find point distances
        d = np.diff(tract, axis=0)
        pointdists = np.hypot(d[:,0], d[:,1], d[:,2])
        dist_threshold = min(pointdists.sum()/len(pointdists),1)
        offset = 0
        for i in xrange(len(pointdists)):
            if pointdists[i] > dist_threshold:
                new_p = (tract[i+1]+tract[i])/2
                new_tract = np.insert(new_tract,i+1+offset,new_p,axis=0)
                offset += 1
        return new_tract
    #
    def returnBundlePoints(bundle):
        in_tractography = tractography.tractography_from_files(bundle)
        tracts = in_tractography.tracts()
        ## sort all tracts direction to positive coordinate of their dominant orientation
        ## then interpolate tracts to make sure they are approximately equally placed
        for i in xrange(len(tracts)):
            tracts[i] = AlignFiber(tracts[i])
            #tracts[i] = InterpolateFiber(tracts[i])
        #
        tracts_last_quarter = [tracts[i][4*len(tracts[i])/5:] for i in xrange(len(tracts))]
        #
        pts = np.vstack(tracts)
        pts_last_quarter = np.vstack(tracts_last_quarter)
        return pts, pts_last_quarter
    #
    def returnBhattCoef(gs_pts,sr_pts):
        mn = np.minimum(gs_pts.min(0), sr_pts.min(0))
        mx = np.maximum(gs_pts.max(0), sr_pts.max(0))
        bins = np.ceil((mx - mn))
        gs_hist = np.array([ np.histogram(gs_pts[:,i], bins=bins[i], density=True, range=(mn[i], mx[i]))[0] for i in xrange(3) ])
        sr_hist = np.array([ np.histogram(sr_pts[:,i], bins=bins[i], density=True, range=(mn[i], mx[i]))[0] for i in xrange(3) ])
        coefs = np.array([ np.sqrt( (gs_hist[i]*sr_hist[i])/(gs_hist[i].sum()*sr_hist[i].sum()) ).sum() for i in xrange(3) ])
        #print(coefs)
        return coefs.mean()
    #####
    [gs_pts, gs_pts_last_quarter] = returnBundlePoints(gs_bundle)
    [sr_pts, sr_pts_last_quarter] = returnBundlePoints(sr_bundle)
    #
    coef = returnBhattCoef(gs_pts,sr_pts)
    print(coef)
    coef_last_quarter = returnBhattCoef(gs_pts_last_quarter,sr_pts_last_quarter)
    print(coef_last_quarter)

In [ ]:
ComputeBhattacharyyaCoeficient(gs_cst_left_fn,sr_cst_left_fn)

In [ ]:
ComputeBhattacharyyaCoeficient(gs_cst_right_fn,sr_cst_right_fn)

In [ ]:
from tract_querier import tract_math, tractography
import numpy as np

In [ ]:
def AlignFiber(tract):
        out_tract = tract
        vecs = np.diff(tract, axis=0)
        avg_vec = np.mean(vecs, axis=0)
        dots = np.array([ np.dot(avg_vec,[1,0,0]), np.dot(avg_vec,[0,1,0]), np.dot(avg_vec,[0,0,1]) ])
        idx = np.argmax(np.abs(dots))
        if (dots[idx] < 0):
            out_tract = tract[::-1]
        return out_tract
#
def InterpolateFiber(tract):
    new_tract = tract
    # find point distances
    d = np.diff(tract, axis=0)
    pointdists = np.hypot(d[:,0], d[:,1], d[:,2])
    dist_threshold = min(pointdists.sum()/len(pointdists),1)
    offset = 0
    for i in xrange(len(pointdists)):
        if pointdists[i] > dist_threshold:
            new_p = (tract[i+1]+tract[i])/2
            new_tract = np.insert(new_tract,i+1+offset,new_p,axis=0)
            offset += 1
    return new_tract
#
def returnBundlePoints(bundle):
    in_tractography = tractography.tractography_from_files(bundle)
    tracts = in_tractography.tracts()
    ## sort all tracts direction to positive coordinate of their dominant orientation
    ## then interpolate tracts to make sure they are approximately equally placed
    for i in xrange(len(tracts)):
        tracts[i] = AlignFiber(tracts[i])
        tracts[i] = InterpolateFiber(tracts[i])
    #
    tracts_last_quarter = [tracts[i][4*len(tracts[i])/5:] for i in xrange(len(tracts))]
    #
    pts = np.vstack(tracts)
    pts_last_quarter = np.vstack(tracts_last_quarter)
    return pts, pts_last_quarter

In [ ]:
[pts, pts_last_quarter] = returnBundlePoints(gs_cst_left_fn)

In [ ]:
import pandas as pd
from ggplot import *

In [ ]:
DF_gs_x = pd.DataFrame(np.transpose([pts[:,1], pts[:,2]]))
DF_gs_x.rename(columns = lambda x: str(x), inplace=True)
DF_gs_x.rename(columns={"0": "X"}, inplace=True) # rename a dfcolumn   
DF_gs_x.rename(columns={"1": "Y"}, inplace=True) # rename a dfcolumn 
DF_gs_x

ggplot(aes(x = 'X', y ='Y'),data=DF_gs_x) + \
    geom_point()

In [ ]:
pts = pts_last_quarter

DF_gs_x = pd.DataFrame(np.transpose([pts[:,1], pts[:,2]]))
DF_gs_x.rename(columns = lambda x: str(x), inplace=True)
DF_gs_x.rename(columns={"0": "X"}, inplace=True) # rename a dfcolumn   
DF_gs_x.rename(columns={"1": "Y"}, inplace=True) # rename a dfcolumn 
DF_gs_x

ggplot(aes(x = 'X', y ='Y'),data=DF_gs_x) + \
    geom_point() \
    + xlim(-150, 100) \
    + ylim(-60, 80)